In [ ]:
import numpy as np
import awkward as ak
import pandas as pd
from collections import defaultdict
from utils import readlinesReversed, Table, Col

In [ ]:
hooks = [
    "SequentialVertexFitter::",
    "rave::VertexFactory::",
    "NewV0Fitter::",
    "genfit::GFRaveVertexFactory::",
    "V0FinderModule::",
]

In [ ]:


def checkLine(line, hooks):
    for s in hooks:
        if s in line:
            return True
    return False

def logExtract(filepath):
    events = []
    event = None
    i = 0
    for line in filter(
        lambda l: checkLine(l, hooks),
        readlinesReversed(filepath)
    ):
        [_, fname, *_, t_str, _] = line.split(" ")
        if "event" in fname:
            if event is not None:
                events.append(event)
                i+=1
            
            event = defaultdict(list)
        
        event[fname].append(float(t_str))
    
    if event is not None:
        events.append(event)

    return events

In [ ]:
eventInfos= logExtract(
    "v0finder-validation.log", 
)

In [ ]:
arr = ak.Array(eventInfos)

In [ ]:
event = "V0FinderModule::event"
fields = (event,) + tuple(filter(lambda field: field != event, arr.fields))
npy = np.hstack(
    tuple(
        ak.to_numpy(
            ak.sum(
                ak.fill_none(
                    arr[field],
                    [],
                    0,
                ),
                axis=-1,
            )
        ).reshape(-1, 1)
        for field in fields
    )
)

In [ ]:
npy[:, 1:] /= npy[:, :1]

In [ ]:
df = pd.DataFrame(data=npy, columns=fields)

In [ ]:
stats = df.describe().T.drop("count", axis="columns")
stats

In [ ]:
colTitles = ("",) +  tuple(stats.columns.tolist())
rowTitles = tuple(stats.index.tolist())
data = stats.to_numpy().tolist()

In [ ]:
Table.defs.append(r"\providecommand{\rtms}[1]{}")
Table.defs.append(r"\renewcommand{\rtms}[1]{\SI{#1}{\milli\second}}")

fmtPercent = lambda val: f"{val:.1%}".replace("%", r"\%")
fmtTime = lambda val: r"\rtms{" f"{val:.1f}" + "}"
rowFmt = (fmtTime,) + 4 * (fmtPercent,)
cols = tuple(
    Col(name=colTitle, rows=rows, colType="r")
    for colTitle, rows in zip(
        colTitles,
        zip(
            *(
                (rowTitle,) + tuple(fmt(val) for val in row)
                for rowTitle, row, fmt in zip(rowTitles, data, rowFmt, strict=True)
            )
        ),
    )
)
table = Table(*cols, colLines=True, rowLines=True)

In [ ]:
print(table.render())

In [ ]:
matrix = [
    [1,2,],
    [3,4,],
]

list(zip(*matrix))